In [ ]:
%load_ext autoreload
%autoreload 2
from eo_tools.S1.core import S1IWSwath
from eo_tools.S1.core import load_dem_coords, lla_to_ecef

from eo_tools_dev.util import show_insar_phi

from eo_tools.S1.util import presum
import numpy as np
import os
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

# Parameter setup

In [ ]:
# change with directory containing your S1 products
data_dir = "/data/S1"
# change with directory containing the results
out_dir = "/data/res/test-rtc"
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

# replace with already downloaded and unzipped products (see the other notebooks to download such products)
primary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1.zip"
secondary_dir = f"{data_dir}/S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814.zip"

# subswath to process
iw = 1
# polarization
pol = "vv"
# DEM upsampling
up = 1.8

min_burst = 4
max_burst = 5

# Process individual bursts

In [ ]:
slc = []
lut = []
dems = []
sigma_ts = []

prm = S1IWSwath(primary_dir, iw=iw, pol=pol)
sec = S1IWSwath(secondary_dir, iw=iw, pol=pol)
overlap = np.round(prm.compute_burst_overlap(2)).astype(int)

for burst_idx in range(min_burst, max_burst + 1):
    # log.info(f"---- Processing burst {burst_idx} ----")

    # compute geocoding LUTs for master and slave bursts
    file_dem = prm.fetch_dem_burst(burst_idx, dir_dem="/data/tmp", force_download=False)
    az_p2g, rg_p2g, _, sigma_t = prm.geocode_burst(
        file_dem, burst_idx=burst_idx, dem_upsampling=up
    )

    # read primary and secondary burst raster
    arr_p = prm.read_burst(burst_idx, True)
    
    dem_coords = load_dem_coords(file_dem=file_dem, upscale_factor=up)
    dem_x, dem_y, dem_z = lla_to_ecef(dem_coords[0], dem_coords[1], dem_coords[2], None)
    # arr_p *= sigma_t

    # radiometric calibration 
    cal_p = prm.calibration_factor(burst_idx, cal_type="beta")
    arr_p /= cal_p

    slc.append(np.abs(arr_p))
    sigma_ts.append(sigma_t)
    lut.append((az_p2g, rg_p2g))
    dems.append(file_dem)

# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 10))
# plt.imshow(np.log(sigma_t[:,::4].clip(1e-8)), interpolation="bilinear", vmin = -5, vmax=0)
# plt.colorbar(fraction=0.046, pad=0.04)

# Stitch bursts in a single image

In [ ]:
from eo_tools.S1.core import stitch_bursts
img = stitch_bursts(slc, overlap)
sim = stitch_bursts(sigma_ts, overlap)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(sim[:,::4][2000:,:1000], interpolation="bilinear", vmin=1, vmax=5, cmap="viridis")
# plt.imshow(sim[:,::4].clip(0, 1), interpolation="none")#, vmin=0, vmax=100)
# plt.imshow(sim[:,::4].clip(0, 200), interpolation="none")#, vmin=0, vmax=100)
plt.colorbar()

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img[:,::4], interpolation="bilinear", vmin = 0, vmax=1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(abs(img)[:,::4], interpolation="bilinear", vmax=1)
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow((abs(img)*sim.clip(1e-10))[:,::4], interpolation="bilinear", cmap="gray", vmin=0, vmax=1)
plt.title("Terrain compensated")
plt.axis('off')
# plt.colorbar(fraction=0.03, pad=0.04)
plt.figure(figsize=(10, 10))
plt.imshow(np.abs(img[:,::4]), interpolation="bilinear", cmap="gray", vmin=0, vmax=1)
plt.title("Original")
plt.axis('off')

plt.figure(figsize=(10, 10))
plt.imshow(sim[:,::4].clip(1e-10), interpolation="bilinear", cmap="gray", vmin=0, vmax=5)
plt.axis('off')
# plt.colorbar(fraction=0.01, pad=0.04)

# Reproject and merge complex interferograms

In [ ]:
from eo_tools.S1.core import resample
import rioxarray as riox
from rioxarray.merge import merge_arrays
from eo_tools.auxils import remove

mlt_az = 1
mlt_rg = 4

off = 0
H = int(overlap / 2)
img0 = img.copy()
img0[sim>0] /= sim[sim>0]
phi_out = presum(img0, mlt_az, mlt_rg).clip(1e-8,1)
naz = sigma_ts[0].shape[0]
list_ifg = []
files_to_remove = []
for i in range(min_burst, max_burst + 1):
    log.info(f"Resample burst {i}")
    az_mst, rg_mst = lut[i - min_burst]
    file_dem = dems[i - min_burst]
    cnd = (az_mst >= H - 4) & (az_mst < naz - H + 4)
    az_mst2 = az_mst.copy()
    rg_mst2 = rg_mst.copy()
    az_mst2[~cnd] = np.nan
    rg_mst2[~cnd] = np.nan

    file_ifg = f"{out_dir}/remap_burst_{i}_ifg.tif"
    files_to_remove.append(file_ifg)

    if i == min_burst:
        off2 = off
    else:
        off2 = off - H
    resample(
        phi_out,
        file_dem,
        file_ifg,
        (az_mst2 + off2) / mlt_az,
        (rg_mst2) / mlt_rg,
        kernel="bicubic",
    )
    if i == min_burst:
        off += naz - H
    else:
        off += naz - 2 * H

    list_ifg.append(riox.open_rasterio(file_ifg))

merged_ifg = merge_arrays(list_ifg)
# merged_ifg = merge_arrays(list_ifg, nodata=0, method="last")
merged_ifg.rio.to_raster(f"{out_dir}/merged_img.tif")
print(files_to_remove)
for fname in files_to_remove:
    remove(fname)

In [ ]:
# ds = riox.open_rasterio('/data/res/test-rtc/remap_burst_5_ifg.tif')
ds = riox.open_rasterio('/data/res/test-rtc/merged_img.tif', masked=True)
# ds[0].plot.imshow(vmin=0, vmax=1)
ds[0].plot.imshow()

# Compute phase

In [ ]:
import xarray as xr
# Finite no data value for TiTiler
nodata = -9999
# avoid metadata being lost in arithmetic opetations
xr.set_options(keep_attrs=True)
ifg = riox.open_rasterio(f"{out_dir}/merged_ifg.tif")
phi = np.arctan2(ifg[1], ifg[0])
phi = phi.fillna(nodata)
phi.attrs["_FillValue"] = nodata
phi.rio.to_raster(f"{out_dir}/merged_phi.tif", nodata=nodata)

# Visualize

In [ ]:
show_insar_phi(f"{out_dir}/merged_phi.tif")